In [11]:
!pip install requests elasticsearch tqdm


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
# Load libraries
import requests
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [34]:
# https://youtu.be/vkTiVwwch6A?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=128
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [152]:
# https://youtu.be/vkTiVwwch6A?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=224
# API code: https://youtu.be/ozCpmkbJNJE?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=359
def llm(prompt):
    from openai import OpenAI
    client = OpenAI() # If api_key here is not defined, OpenAI will look into os.environ.get('OPENAI_API_KEY')
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [38]:
# https://youtu.be/1lgbR5wMvsI?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=214
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [39]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
# https://youtu.be/1lgbR5wMvsI?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=312
es_client = Elasticsearch('http://localhost:9200')

#es_client.info() # same information as it is in bash terminal: curl localhost:9200

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

# Create indices
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
# Index dataset
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:31<00:00, 30.51it/s]


In [42]:
query = 'How do I execute a command in a running docker container?'

In [126]:
# https://youtu.be/1lgbR5wMvsI?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=569
def elastic_search(query, size = 5, fields = ["question^3", "text", "section"], course = "data-engineering-zoomcamp", incl_score = False):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    if course is not None:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": course
            }
        }

    response = es_client.search(index=index_name, body=search_query)

    if incl_score:
        result_docs = [{'_source': i['_source'], '_score': i['_score']} for i in response['hits']['hits']]
    else:
        result_docs = [i['_source'] for i in response['hits']['hits']]
    
    return result_docs

In [127]:
elastic_search(query)

[{'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
  'section': 'Module 1: Docker and Terraform',
  'question': 'PGCLI - running in a Docker container',
 

In [136]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt) # Need OpenAI's API code to run llm() function.
    return answer

In [ ]:
answer = rag(query)

In [ ]:
print(answer)

## Homework
https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

In [85]:
query = 'How do I execute a command in a running docker container?'

In [128]:
elastic_search(query, size = 1, fields = ["question^4", "text"], course = None, incl_score=True)

[{'_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  '_score': 84.050095}]

In [129]:
q4 = elastic_search(query, size = 3, fields = ["question^4", "text"], course="machine-learning-zoomcamp")
q4[2]

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

In [131]:
# q5
question = query
text = q4

context = f"""
Q: {question}
A: {text}
""".strip()

In [132]:
context

'Q: How do I execute a command in a running docker container?\nA: [{\'text\': \'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\\ndocker run -it --entrypoint bash <image>\\nIf the container is already running, execute a command in the specific container:\\ndocker ps (find the container-id)\\ndocker exec -it <container-id> bash\\n(Marcos MJD)\', \'section\': \'5. Deploying Machine Learning Models\', \'question\': \'How do I debug a docker container?\', \'course\': \'machine-learning-zoomcamp\'}, {\'text\': "You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\\nHrithik Kumar Advani", \'section\': \'5. Deploying Machine Learning Mod

In [133]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [104]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 878.7 kB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.2 MB/s eta 0:00:00m eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [134]:
len(prompt_template)

1883

In [153]:
#q6
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt_template))

439